<a href="https://colab.research.google.com/github/componavt/wd_book/blob/master/programming_tasks/natural_disasters/ruleq/Ruleq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# This program draws volcanoes and the areas only of those disasters that occur at a given distance and number of volcanoes around it between the earthquake and the volcano according to the CSV files query_eq.csv and query_vul.csv

Эта программа рисует вулканы и области землетрясений, которые происходят на заданном расстоянии и при определенном количестве вулканов вокруг них согласно CSV-файлам query_eq.csv и query_vul.csv.


**Для работы программы необходимо два файла query_eq.csv и query_vul.csv, сгенерированные при помощи SPARQL-запросов:** https://w.wiki/5$ih и https://w.wiki/5$iq

In [1]:
import folium
import csv
import numpy as np
from scipy.spatial import KDTree
import random

!wget https://raw.githubusercontent.com/componavt/wd_book/master/programming_tasks/natural_disasters/vuleq_connect/volcanoes_2023.csv
!wget https://raw.githubusercontent.com/componavt/wd_book/master/programming_tasks/natural_disasters/vuleq_connect/earthquakes_2023.csv

--2024-07-09 18:24:41--  https://raw.githubusercontent.com/componavt/wd_book/master/programming_tasks/natural_disasters/vuleq_connect/volcanoes_2023.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76165 (74K) [text/plain]
Saving to: ‘volcanoes_2023.csv’

volcanoes_2023.csv  100%[===================>]  74.38K  --.-KB/s    in 0.02s   

2024-07-09 18:24:41 (3.30 MB/s) - ‘volcanoes_2023.csv’ saved [76165/76165]

--2024-07-09 18:24:41--  https://raw.githubusercontent.com/componavt/wd_book/master/programming_tasks/natural_disasters/vuleq_connect/earthquakes_2023.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [2]:
map_of_ruleq = folium.Map()
volcano_coords = []  # Список для хранения координат вулканов
eq_coords = []  # Список для хранения координат землетрясений

# Параметры для фильтрации
max_volcano_eq_distance_km = 100  # Максимальное расстояние между вулканом и землетрясением в километрах

# Преобразуем расстояние в километрах в градусы (приблизительно)
max_volcano_eq_distance = max_volcano_eq_distance_km / 111

# Минимальное количество вулканов вокруг землетрясения
min_volcanoes_around_eq = 10

#Радиус круга для рисования
radius_meters = max_volcano_eq_distance * 111000

# Открытие файла CSV и его обработка для вулканов
with open("volcanoes_2023.csv", encoding='utf-8', newline='') as csvfile:
    reader1 = csv.DictReader(csvfile, delimiter=";")
    for row in reader1:
        cords = row['location'].replace("Point(", "").replace(")", "").split()
        volcano_coords.append((float(cords[0]), float(cords[1])))  # Координаты вулкана

# Открытие файла CSV и его обработка для землетрясений
with open("earthquakes_2023.csv", encoding='utf-8', newline='') as csvfile:
    reader2 = csv.DictReader(csvfile, delimiter=";")
    for row in reader2:
        cords = row['location'].replace("Point(", "").replace(")", "").split()
        eq_coords.append((float(cords[0]), float(cords[1])))  # Координаты землетрясения

# Создание KDTree для вулканов
tree = KDTree(volcano_coords)

# Поиск ближайших вулканов к каждому землетрясению
for eq_coord in eq_coords:
    nearby_volcano_indices = tree.query_ball_point(eq_coord, r=max_volcano_eq_distance)  # Ищем вулканы в радиусе
    if len(nearby_volcano_indices) >= min_volcanoes_around_eq:  # Проверка минимального количества вулканов
        fill_color = "#{:06x}".format(random.randint(0, 0xFFFFFF))  # Генерация случайного цвета
        folium.Circle(radius=radius_meters, location=(eq_coord[1], eq_coord[0]),
                      tooltip='Вулканы вблизи землетрясений', color="blue", fill=True,
                      fill_color=fill_color, fill_opacity = 0.4).add_to(map_of_ruleq)

with open("volcanoes_2023.csv", encoding='utf-8', newline='') as csvfile:
    reader1 = csv.DictReader(csvfile, delimiter=";")
    for row in reader1:
        cords = row['location'].replace("Point(", "").replace(")", "").split()
        volcano_coords.append((float(cords[0]), float(cords[1])))
        folium.Circle(radius=500, location=(float(cords[1]), float(cords[0])), popup=row['volcanoLabel'],
                      tooltip='Вулкан', color="black").add_to(map_of_ruleq)

map_of_ruleq.save('карта_с_землетрясениями_вблизи_вулканов.html')
map_of_ruleq